### Experiments with Large Language Models

- fine tune T5 for qa
- fine tune T5 for span corruption

In [20]:
# General imports
import pandas as pd
import numpy as np

#  PyTorch imports
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

# Transformers imports
from transformers import T5Tokenizer, T5ForConditionalGeneration


#### Data

In [21]:
wiki = []
with open('data/wiki.txt', 'r', encoding='utf8') as f:
    for line in f:
        wiki.append(line)

wiki = pd.DataFrame(wiki, columns=['text'])
wiki.head()

,text
0,Khatchig Mouradian. Khatchig Mouradian is a jo...
1,Jacob Henry Studer. Jacob Henry Studer (26 Feb...
2,"John Stephen. Born in Glasgow, Stephen became ..."
3,Georgina Willis. Georgina Willis is an award w...
4,Stanley Corrsin. Corrsin was born on 3 April 1...


In [22]:
# Read tsv file
birth_places_train = pd.read_csv('data/birth_places_train.tsv', sep='\t') 
birth_places_train.head()

,Where was Khatchig Mouradian born?,Lebanon
0,Where was Jacob Henry Studer born?,Columbus
1,Where was John Stephen born?,Glasgow
2,Where was Georgina Willis born?,Australia
3,Where was Stanley Corrsin born?,Philadelphia
4,Where was Eduard Ender born?,Rome


In [23]:
birth_places_test = pd.read_csv('data/birth_places_test.tsv', sep='\t')
birth_places_test.head()

,Where was Bryan Dubreuiel born?,Atlanta
0,Where was Ralf Wadephul born?,Berlin
1,Where was Joseph Baggaley born?,England
2,Where was Sandhya Sanjana born?,Mumbai
3,Where was Alfred Mele born?,Detroit
4,Where was Murray Esler born?,Geelong


In [24]:
class BirthPlaceDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.df_values = df.values
        
    def __len__(self):
        return len(self.df_values)
    
    def __getitem__(self, idx: int):
        return self.df_values[idx]

In [25]:
train_dataset = BirthPlaceDataset(birth_places_train)
test_dataset = BirthPlaceDataset(birth_places_test)

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [28]:
## Load t5 small model for question answering
model = T5ForConditionalGeneration.from_pretrained('t5-small', return_dict=True)

Downloading: 100%|██████████| 1.21k/1.21k [00:00<00:00, 1.22MB/s]
Downloading: 100%|██████████| 242M/242M [00:49<00:00, 4.88MB/s] 
c:\Users\Popescu Andrei\anaconda3\envs\ml\lib\site-packages\torch\serialization.py:953: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:77.)
  obj = cast(Storage, torch.UntypedStorage(nbytes))


In [29]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')

TypeError: 'NoneType' object is not callable